#### Knn From Scratch:

The goal of this exercise to implement the knn algorithm from scratch for both regression and classification problem.
In addition to this, the cross validation and nested cross validation are also implemented from scratch and compare them with the built in scikit-learn libary. Thus, knn_class is defined. For classification problem, method- knn_classifer_prediction() should be called and for regression problem, method- knn_regression_prediction() should be called. The algorithm is implemented as following steps-wise:

   1.Find the distance(Eculidan is used) between two data points(instances)

   2.Sort the distance. Take the 'first k most closest neighbours' of the test instance in the training data set.
      
   For classification: Find the most frequent neighbour from the  'k most closet neighbours'  and labels that test instance to that neighbour(class) 
        
   For Regression:  Find the average of  target varaibles of the 'k most closest neigbours' and used as predicted value for that test instance 

In [2]:
#import necessary libraries
import numpy as np
import pandas as pd
import random
from random import randrange
import operator

In [17]:
# define Knn class
class knn_class:
    
    """ Both classification and regression problem are solved with this code."""
    
    # calculates eculidan distance between two data instances
    def eculidan_dist(self, data1, data2, length):
        distance = 0
        # loop through all variables
        for x in range(length):
            distance += np.power((data1[x]- data2[x]),2)
        return np.sqrt(distance)
    
    
    
    
    # finds k nearest  neighbors  to test instance using the ecludian distance 
    def get_neighbours(self, trainingSet, testInstance, k):
        distance = []
        length =  len(testInstance) -1 # all variables except target variable
        
        # find distance  between test instance to all training instances
        for x in range(len(trainingSet)):
            dist =  self.eculidan_dist(trainingSet[x], testInstance, length) # call eculidan_dist function
            distance.append((trainingSet[x], dist)) 
        
        # sort the distance in ascending order
        distance.sort(key=operator.itemgetter(1))
        #print(distance)
    
        #store the  k neighbours
        neighbours = []
        for x in  range(k):
            neighbours.append(distance[x][0])
        return neighbours
    
    
    
    """ ======== For classification problem, start from here ======= """
    # finds class that test instance belongs to. This is done by finding the  the most frequent class in the k neigbours
    def get_response(self, neighbors):
        classVotes = {}
        for x in range(len(neighbors)):
            response = neighbors[x][-1] # take last attribute(target variable) 
            if response in classVotes:
                classVotes[response] +=1
            
            else:
                classVotes[response] = 1
    
        # sorts classes with higest values
        sortedVotes = sorted(classVotes.items(),key=operator.itemgetter(1), reverse=True) #sorting based on max count
        return  sortedVotes[0][0] # take first neighbour
    
    
    
    
    # generates prediction for the given test set data
    def knn_classifer_prediction(self, testSet, trainingSet,  k):
        predictions = []
        actual = []
        for x in range(len(testSet)):
            neighbors = self.get_neighbours(trainingSet, testSet[x], k )
            response = self.get_response(neighbors)
            predictions.append(response)
            actual.append(testSet[x][-1])
        return predictions, actual
        

    # find the prediction accuracy
    def get_accuracy(self, testSet, predictions):
        correct = 0.0
        
        #compare the testSet target varibles with the prediction class
        # and count the correct number of predictions
        for x in range(len(testSet)):
            if testSet[x][-1] is predictions[x]:
                correct +=1
        
        return correct/ float(len(testSet))
        
  

    """============ For Regression problem start from here ==============="""      
          
    # finds average of the k neighbors target variables
    def get_average(self, neighbors):
        ave_value = 0
        total_neig = len(neighbors)
        for x in range(total_neig):
            ave_value += neighbors[x][-1] # take last attributes (target varibales)
        return round(ave_value/total_neig, 2) # average the values
    
    
    
    # generates prediction of the given testset
    def knn_regression_prediction(self, testSet, trainingSet,  k):
        predictions = []
        for x in range(len(testSet)):
            neighbors = self.get_neighbours(trainingSet, testSet[x], k ) # call get_neighbours function
            print(neighbors)
            predict_val = self.get_average(neighbors) # call get_average function
            predictions.append(predict_val) # sum the value
        return predictions
   
    

##### Dataset:
The iris flower dataset is used  for the classfication  problem. It consists of 150 data instances of three class each 
of 50 instances. The dataset has  4 predicted attributes and the class. In the dataset, the labels/classes are placed in order form, which means that there no randomization of the class. Thus, the data need to shuffle.

In [18]:
#load  iris dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
col_name = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width','Flower_Name']
df = pd.read_csv(url, header=None, delimiter=',', names=col_name)
iris_df = df.sample(frac=1).reset_index(drop=True) # shuffle the data and reset index
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,Flower_Name
0,6.0,2.7,5.1,1.6,Iris-versicolor
1,5.5,3.5,1.3,0.2,Iris-setosa
2,5.1,2.5,3.0,1.1,Iris-versicolor
3,5.6,2.5,3.9,1.1,Iris-versicolor
4,6.3,3.4,5.6,2.4,Iris-virginica


In [19]:
#provides the statistical descriptive 
iris_df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [20]:
# spilt data into train set and test set
def train_test_split(data_set, split_ratio):
    trainingSet = []
    testSet = []
    
    for x in range(len(data_set)):
        if random.random() < split_ratio:
            trainingSet.append(data_set[x])
        else:
            testSet.append(data_set[x])
    return trainingSet, testSet


In [21]:
#call the train_test_split function
data_set = iris_df.values.tolist() # convert to python list 
split_ratio = 0.70
#random.seed(30) # split into  same data every time code is executed 
train_set, test_set = train_test_split(data_set, split_ratio)
print('Test size: ',len(test_set), ' and Train size: ', len(train_set))

Test size:  44  and Train size:  106


In [22]:
#check the classification problem 

knn_object =  knn_class()
k = 5
predict, actual = knn_object.knn_classifer_prediction(test_set, train_set, k) #call knn classification method

#Observed class and predicted class
print('Observed class and predicted class:')
for x in range(len(predict)):
    print(' Predicted =', predict[x], ',    Actual =',  actual[x])

#find accuracy
accu_score = knn_object.get_accuracy(test_set, predict)
print('\nAccuracy is {0:.2f}'.format(accu_score))


Observed class and predicted class:
 Predicted = Iris-setosa ,    Actual = Iris-setosa
 Predicted = Iris-versicolor ,    Actual = Iris-versicolor
 Predicted = Iris-virginica ,    Actual = Iris-virginica
 Predicted = Iris-versicolor ,    Actual = Iris-versicolor
 Predicted = Iris-setosa ,    Actual = Iris-setosa
 Predicted = Iris-versicolor ,    Actual = Iris-versicolor
 Predicted = Iris-setosa ,    Actual = Iris-setosa
 Predicted = Iris-virginica ,    Actual = Iris-versicolor
 Predicted = Iris-versicolor ,    Actual = Iris-versicolor
 Predicted = Iris-versicolor ,    Actual = Iris-versicolor
 Predicted = Iris-virginica ,    Actual = Iris-virginica
 Predicted = Iris-virginica ,    Actual = Iris-versicolor
 Predicted = Iris-versicolor ,    Actual = Iris-versicolor
 Predicted = Iris-versicolor ,    Actual = Iris-versicolor
 Predicted = Iris-versicolor ,    Actual = Iris-versicolor
 Predicted = Iris-versicolor ,    Actual = Iris-versicolor
 Predicted = Iris-setosa ,    Actual = Iris-setosa

In [25]:
#check for regression problem
knn_object =  knn_class()
k=3
test= [[1,2,3,4,2]]
train= [[3.4,2,1,4,6], [5,2,1,4,5], [2,2.5,3,1,4], [5,2,3,4,7]]
predict = knn_class().knn_regression_prediction(test, train, k) # call regresion method

print('preditected value is: ', predict)


[[3.4, 2, 1, 4, 6], [2, 2.5, 3, 1, 4], [5, 2, 3, 4, 7]]
preditected value is:  [5.67]


                                       ====== Cross Validation =========
The two functions are used for Cross Validation i.e 1) cross_validation_split() and 2) cross_validation() . The first function divides the dataset into 'n_folds' and the second function is used to  evaluate the model performance.  For separating the test and train dataset, the python list indexing is used.          
                    

In [9]:
# split data for cross validation
def cross_validation_split(dataset, folds):
    fold_size = int(len(dataset)/folds) # size  of each fold
    #print(fold_size)
    dataset_split =  []
    dataset_copy = dataset.copy() #duplicate the dataset, so that it is used for generating index later 
    
    
    #assign data instance to each folds
    for x in range(folds):
        each_fold_data = []
        while len(each_fold_data) < fold_size:
            index = randrange(0,len(dataset_copy)) # generate random number from 0 to size of dataset
            each_fold_data.append(dataset_copy.pop(index)) # append  each popped data to each fold
        dataset_split.append(each_fold_data) # append each fold to data_split
        
        
    #check if the items are remain in list (dataset_copy) and assign to the last fold
    if dataset_copy:
        for x in range(len(dataset_copy)):
            dataset_split[-1].append(dataset_copy[x])
       #dataset_split[-1].extend(dataset_copy)
    #print(dataset_copy)
    
    return dataset_split
        
    
    
    
    

In [256]:
# Cross Validation  Method
import random
def cross_validation(dataset, n_folds, k):
    random.seed(2)
    data_split = cross_validation_split(dataset,n_folds) # call cross_validation_split function
    accu_cross_val = []
    
    #take single fold as test data and remaisnig(n-1) as train data (list indexing is used)
    for index, item  in enumerate(data_split):
        test = data_split[index] # for test data
        train = data_split[0:index]+ data_split[index+1:] # for train data
        train = [instance for item in train for instance in item ] # change 3d into 2d list
            
        prediction, actual = knn_object.knn_classifer_prediction(test, train, k) #gives prediction and actual class
        accuracy = knn_object.get_accuracy(test, prediction) #find the accuracy
        #print(accuracy)
        accu_cross_val.append(accuracy)
        #print(accuracy,\n')
        
        
    #average the score from n_folds
    score = np.mean(accu_cross_val) 
    return round(score,2)
        

In [251]:
#check  cross_validation
cross_validation(train_set,10, 5)

0.95

In [257]:
# find optimal k for knn using 10-fold cross validation
optimal_k = 0
high_score = 0
     
#iterates over number of k in KNN    
for k in range(1,20):
    score_cv =  cross_validation(train_set,10, k) #call crossvalidation function
    #print('{0:.2f}'.format(score_cv))
    
    #check k of high score
    if score_cv > high_score:
        high_score = score_cv
        optimal_k = k
    
print('Optmal_k is {0} and high_score is {1:.2f}'.format(optimal_k, high_score)) 
    
    
    
       
        

Optmal_k is 9 and high_score is 0.98


                        ========= Nested Cross Validation ===========
Nested Cross Validation has two cross valdiation loops i.e outer and inner loop. Outer loop is used for model perfomance evaluation whereas inner loop is used for finding/tunning the model optimal parameter, for example, 
k in Knn.

In [10]:
# define nested cross validation function
import random
def nested_crossValidation(dataset, n_folds):
    random.seed(32)
    
    #split the dataset into n_folds
    data_split = cross_validation_split(dataset, n_folds)  #call split function
    predictin_accuracy = []
    
    ''' Outer CV start here'''
    #outer loop use for model performance evaluation
    for index, item in enumerate(data_split):
        test = data_split[index] # for test data
        train = data_split[0:index]+ data_split[index+1:] # for train data
        train = [instance for item in train for instance in item ] # change 3d into 2d list
        
        
        
        high_score = 0
        optimal_k = 0
        
        #find the optimal value of k
        for k in range(1,20):
            accu_inner_cross = [] # stores accuracy of test fold of inner CV
            
            
            '''Inner CV start here, train set of Outer CV is used as data set in inner CV'''
            
            # train data of outer CV  splits into futher folds
            data_split_1 = cross_validation_split(train, n_folds) 
            
            #inner loop user for finding the optimal k in KNN
            for index, item in enumerate(data_split_1):
                test_1 = data_split_1[index] # for test data
                train_1 = data_split_1[0:index]+ data_split_1[index+1:] # for train data
                train_1 = [instance for item in train_1 for instance in item ] # change 3d into 2d list
            
                prediction, actual = knn_object.knn_classifer_prediction(test_1, train_1, k ) # call knn_classifer_prediction method
                accuracy = knn_object.get_accuracy(test_1, prediction) # call get_accuracy method
                accu_inner_cross.append(accuracy)
           
            score =  np.mean(accu_inner_cross)
            #print(score)
            
            #check for optimal k of high score 
            if score > high_score:
                high_score= score
                optimal_k =k
        

        print('Optimal_k is:',optimal_k)
        
        #use the optimal k and evaluate the model prediction
        prediction, actual = knn_object.knn_classifer_prediction(test, train, optimal_k )
        accuracy = knn_object.get_accuracy(test, prediction)
        predictin_accuracy.append(accuracy)
        
    return  round(np.mean(predictin_accuracy),2)
    

In [254]:
#check nested cross validation
n_folds = 10
dataset = train_set
nested_crossValidation(dataset, n_folds)

Optimal_k is: 7
Optimal_k is: 14
Optimal_k is: 12
Optimal_k is: 11
Optimal_k is: 10
Optimal_k is: 8
Optimal_k is: 9
Optimal_k is: 12
Optimal_k is: 9
Optimal_k is: 17


0.96

               ========= Comparing with scikitlearn library ============
               
To compare implemented knn  classifer with the scikitlearn Knn classifer, the same  training and testing dataset 
should be in both. Already in the knn scratch implementation, the data is python list form. So it should convert 
into pandas dataframe  because it is more convient to seprate  from panda dataframe into  input features and  target 
class. The reason doing this is the knn algorithm in scikitlearn uses separate input features and target lables 

In [11]:
# import necessary libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import  cross_val_score, KFold
from sklearn import metrics


In [12]:
#Convert the data into pandas data frame

train = pd.DataFrame(np.array(train_set), columns=iris_df.columns)
# train.iloc[:,0:4] = train.iloc[:,0:4].apply(pd.to_numeric)
test = pd.DataFrame(np.array(test_set), columns=iris_df.columns)
# test.iloc[:,0:4] = train.iloc[:,0:4].apply(pd.to_numeric)
print('train size:', train.shape)
train.head()


train size: (112, 5)


,sepal_length,sepal_width,petal_length,petal_width,Flower_Name
0,5.0,3.5,1.6,0.6,Iris-setosa
1,5.7,4.4,1.5,0.4,Iris-setosa
2,5.5,2.4,3.8,1.1,Iris-versicolor
3,7.7,3.8,6.7,2.2,Iris-virginica
4,5.5,2.6,4.4,1.2,Iris-versicolor


In [13]:
print('test size:', test.shape)
test.head()

test size: (38, 5)


,sepal_length,sepal_width,petal_length,petal_width,Flower_Name
0,6.5,3.2,5.1,2.0,Iris-virginica
1,6.2,2.2,4.5,1.5,Iris-versicolor
2,6.0,3.4,4.5,1.6,Iris-versicolor
3,4.9,3.1,1.5,0.1,Iris-setosa
4,5.6,3.0,4.1,1.3,Iris-versicolor


In [14]:
#define knn classifer
def knn_function(x_train, y_train, x_test, y_test, k):
    knn = KNeighborsClassifier(n_neighbors=k) # initiate knn classier
    knn.fit(x_train,y_train) # train with data
    prediction = knn.predict(x_test) # get prediction
    print('Prediction: \n {}'.format(prediction))
    #a = knn.score(x_test,y_test)
    #print(a)
    accu_score= metrics.accuracy_score(y_test, prediction)
    return accu_score


In [15]:
# split data into input features and target class
x_train, y_train = train.iloc[:,0:4], train.iloc[:,-1]
x_test, y_test = test.iloc[:,0:4], test.iloc[:,-1]

k= 5 #since we use k= 5 in knn scratch

accu_score = knn_function(x_train, y_train, x_test, y_test, k) # call knn_function 
print('\n Accuracy is: {0:.2f}'.format(accu_score))

Prediction: 
 ['Iris-virginica' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor'
 'Iris-virginica' 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor'
 'Iris-setosa' 'Iris-virginica' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor' 'Iris-setosa'
 'Iris-virginica' 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor'
 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica'
 'Iris-virginica' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica' 'Iris-virginica'
 'Iris-virginica' 'Iris-versicolor']

 Accuracy is: 0.95


In [243]:
#Cross validation using KFolds
knn = KNeighborsClassifier(n_neighbors=5)
kflod = KFold(n_splits=10)
x_train, y_train = train.iloc[:,0:4], train.iloc[:,-1] # separates input features and target variable
score = cross_val_score(knn, x_train, y_train, cv=kflod, scoring='accuracy')
print('Cross Validation score is {0:.2f}'.format(score.mean()))


Cross Validation score is 0.97


In [268]:
#Nested Cross validation

kflod = KFold(n_splits=10) #divides data into 10 folds
data, target = train.iloc[:,0:4], train.iloc[:,-1] # separates input features and target variable

outer_loop_acc = [] # stores accuracy of each folds in outer CV

"""Outer CV start here"""
#Provides train/test indices to split data in train/test sets
for  train_index, test_index in kflod.split(data):
    X_train, X_test = data.iloc[train_index], data.iloc[test_index] 
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    #print(len(X_train), len(X_test))
    
    
    
    bestAccuracy =0
    optimal_k = 0
    
    #iterates over k = (1,20) in KNN
    for k in range(1,20): 
        inner_loop_accu = [] # store accuracy  of each fold in inner CV
    
        """Inner CV start here, train set of outer CV is used as dataset in inner CV"""
        
        #Provides train/test indices to split data in train/test sets
        for  train_index, test_index in kflod.split(X_train):
            X_train_inner, X_test_inner = X_train.iloc[train_index], X_train.iloc[test_index] 
            y_train_inner, y_test_inner = y_train.iloc[train_index], y_train.iloc[test_index]
            #print(len(X_train_inner),len(X_test_inner))
        
            knn= KNeighborsClassifier( n_neighbors= k) # create Knn classifer
            knn.fit(X_train_inner,y_train_inner) # train the classifier
            prediction = knn.predict(X_test_inner) # get response from test data
            accuracy = metrics.accuracy_score(y_test_inner, prediction) # calculates accuracy
            inner_loop_accu.append(accuracy)
        
        score = np.mean(inner_loop_accu) # average the
        #print('k', k)
        #print(score)
        
        #check best accuracy
        if score > bestAccuracy:
            bestAccuracy = score
            optimal_k = k
    
    print('optimal_k is: {0}'.format(optimal_k))
    
    # use optimal k for outer CV 
    knn= KNeighborsClassifier( n_neighbors= optimal_k) # create Knn classifer
    knn.fit(X_train, y_train) # train the classifier
    prediction = knn.predict(X_test) # get response from test data
    accuracy = metrics.accuracy_score(y_test, prediction) # calculates accuracy
    outer_loop_acc.append(accuracy)

print('\nModel accuracy is: {0:.2f}'.format(np.mean(outer_loop_acc)))

    

optimal_k is: 8
optimal_k is: 10
optimal_k is: 5
optimal_k is: 18
optimal_k is: 3
optimal_k is: 16
optimal_k is: 16
optimal_k is: 3
optimal_k is: 3
optimal_k is: 3

Model accuracy is: 0.95


##### Conclusion: 
After analysing the results, the implemented Knn classifer works fine as like scikit-learn knn classifer. The cross validation and  nested cross validation work same as scikit-learn. There is almost the same result both in scikit-learn and implemented knn classifer.
    